In [4]:
import os
import re
import sys
from PyPDF2 import PdfReader, PdfWriter, PdfMerger
import platform
import unicodedata


In [5]:
hostname = platform.node()
print(hostname)

if hostname == "T_800_Model_101":
   directory = "D:/Dropbox/"

if hostname == "ECON-STAFF-ASSOC-2022IMAC-H4TJ80ZFQ6X9-2":
   directory = "/Users/gr2757/Dropbox/"

ECON-STAFF-ASSOC-2022IMAC-H4TJ80ZFQ6X9-2


In [6]:
input_directory = directory + "/global_taxation/Inputs/"
year_page_gap = [[2004, 7], [2005, 7], [2006, 7], [2007, 7], [2008, 7], [2009, 7],[2010, 7], [2011, 7], [2012, 7], [2013, 8], 
                 [2014, 8], [2015, 7],[2016, 8], [2017, 8], [2018, 8], [2019, 9], [2020, 8], [2021, 10],[2022, 8], [2023, 10]]


for fileName in os.listdir(input_directory):
    if fileName.endswith(".pdf"):
        full_file_name = input_directory +fileName
        pdfFile = PdfReader(full_file_name)

        data_year = str(fileName[-8:-4])

        for sublist in year_page_gap:
            if sublist[0] == int(data_year):
                pageOffset = int(sublist[1])
                break

        #set the index
        pdfInd = 0

        #setting index
        page = pdfFile.pages[pdfInd]
        pageContent = page.extract_text()
        #intitalize dictionary 
        countries = {}

        # Find the table of contents
        while not re.search(r'Contents\n', pageContent, re.IGNORECASE):
            pdfInd =  pdfInd + 1
            page = pdfFile.pages[pdfInd]
            pageContent = page.extract_text()
            start_index = pdfInd
        
        pdfInd = 0
        pageContent = page.extract_text()
        single_country = ""
        last_country_index = ""
        while not re.search(r"London", pageContent, re.IGNORECASE):
            pdfInd =  pdfInd + 1
            page = pdfFile.pages[pdfInd]
            pageContent = page.extract_text()
            end_index = pdfInd

        full_PageCountry = [] #intitalizing the list, this is to be for the whole document not just per page.

        for i in range(start_index,end_index):
            page = pdfFile.pages[i]
            pageContent = page.extract_text()
            contentsPage = re.split(r'\.{2,}', pageContent) #splitting by the "......."
            contentsPage = [s.replace("\n", "") for s in contentsPage] #remove the new line characters to clean up the list
            contentsPage = [re.sub(r'\(european\s*union\s*member\s*state\)', '', s, flags=re.IGNORECASE) for s in contentsPage] #clean up this extra label
            contentsPage = [re.sub(r'Special Administrative Region (SAR) of China', '', s, flags=re.IGNORECASE) for s in contentsPage] #clean up this extra label
            contentsPage = [s.replace("Y emen", "Yemen") for s in contentsPage] #clean up this extra label
            contentsPage = [s.replace("Ireland, Republic of", "Ireland") for s in contentsPage] #clean up this extra label


            #the end of the list will be zimbabwe
            for i in range(len(contentsPage)):
                if re.search(r"Zimbabwe", contentsPage[i]):
                    last_country_index = i + 3

                #for some reason there is this hidden text in the pdf that shows up here, this is code to remove it
                if re.search(r"WWCT", contentsPage[i]):
                    temp_list = contentsPage[i].split("WWCT")
                    contentsPage[i] = temp_list[0]

            if last_country_index:
                del contentsPage[last_country_index:] #removing all items after the Last country

            for ind, country in enumerate(contentsPage):
                if ('Contents' in country or 
                    'Ernst &' in country):
                    continue
                
                pageCountry = re.split(r'(\d+|\D+)', country)
                pageCountry = [item.strip() for item in pageCountry if item]
                pageCountry = [value for value in pageCountry if value]


                #making one big list of the whole table of contents
                full_PageCountry.append(pageCountry)

            
        contents_list = []

        # This while loop is for dealing with single items in a list due being at the top of the page
        i = 0
        while i < len(full_PageCountry):
            # Check if the current list has one numeric element
            if len(full_PageCountry[i]) == 1 and full_PageCountry[i][0].isdigit():
                # Check if the next list has one non-numeric element
                if i + 1 < len(full_PageCountry):
                    # Merge the two elements
                    merged = [full_PageCountry[i][0], full_PageCountry[i + 1][0]]
                    contents_list.append(merged)
                    # Skip the next element
                    i += 2
                    continue
            
            # If no merging is needed, just append the current list
            contents_list.append(full_PageCountry[i])
            i += 1
        #removing empty items in the list
        for sublist in contents_list:
            if not sublist:
                contents_list.remove(sublist)
        
        #converting to dictionary
        for item in contents_list:
            page, country = item
            countries[country] = page

        
        updated_country_page_dict = {}

        items = list(countries.items())
        
        # Iterate through the list of tuples
        for i in range(len(items) - 1):
            country, page = items[i]
            next_page = items[i + 1][1]
            updated_country_page_dict[country] = next_page


        # removing countries and pages that break the page length spans
        delete_keys = [key for key in updated_country_page_dict if re.search(r"(Other jurisdictions chapter|Emerging markets chapter)", key, re.IGNORECASE)]

        for key in delete_keys:
            del updated_country_page_dict[key]

        # Removing things from the table of contents that are not countries
        while updated_country_page_dict and not next(iter(updated_country_page_dict)).startswith('A'):
            first_key = next(iter(updated_country_page_dict))  # Get the first key
            del updated_country_page_dict[first_key]  # Delete the first key-value pair

        values = (list(updated_country_page_dict.values()))
        page_lengths = []

        #calculating the length of pages
        for i in range(len(values) - 1):
            page_lengths.append(int(values[i + 1]) - int(values[i]))
        
        #some pages are cut short so adding another page for padding
        for index, item in enumerate(page_lengths):
            if item == 1:
                page_lengths[index] = 2 
                #print(page_lengths[index])

        final_list = list(updated_country_page_dict.keys())

        #removing the item after the last country as it was only used for page lengths
        final_list = final_list[:-1]
        values = values[:-1]

        #normalize unicode
        for i in range(len(final_list)):
            final_list[i] = unicodedata.normalize("NFC", final_list[i])

        
        #standardizing names across all years
        pattern_replacements = {
            r"Korea.*": "South Korea",
            r"Palest.*": "Palestine",
            r"Macau.*": "Macau",
            r"Hong Kong.*": "Hong Kong",
            r"US Virgin Islands": "U.S. Virgin Islands",
            r"Bonaire.*": "Bonaire, St. Eustatius and Saba (BES-Islands)",
            r"Eswatini.*": "Eswatini",
            r"Türkiye" : "Turkey",
            r"Serbia, Republic of" : "Serbia",
            r"Côte d’Ivoire" : "Côte D’Ivoire (Ivory Coast)",
            r"Macedonia.*" : "North Macedonia",
            r"\bMontenegro, Republic of\b": "Montenegro"
        }

        # Iterate over the final_list and apply the replacements
        for i in range(len(final_list)):
            for pattern, replacement in pattern_replacements.items():
                if re.search(pattern, final_list[i]):
                    final_list[i] = replacement
        
        #I want to make sure Hong Kong and Macau are done being cleaned before doing China, so I'm doing this in a seperate loop
        for i in range(len(final_list)):
            if re.search(r"China.*", final_list[i]):
                final_list[i] = "China"

        countries = {key: {"page": page, "length": length} for key, page, length in zip(final_list, (values), page_lengths)}


        
        #this is for making the pdfs 

        for country in countries:
            pdfMerge = PdfMerger()
            pageInfo = countries[country]
            if not os.path.exists(f'Outputs/{country}'): # making folders in the loop for each country
                os.makedirs(f'Outputs/{country}') #ignore if it's there

            for ind in range(0, pageInfo['length']):
                pdfWrite = PdfWriter()

                full_page = int((pageInfo['page']))
                temp = full_page + ind + pageOffset - 1
                pdfWrite.add_page(pdfFile.pages[temp])
                with open(f'{country}{ind}.pdf', "wb") as pdfOutput:
                    pdfWrite.write(pdfOutput)
            for ind in range(0, pageInfo['length']):
                pdfMerge.append(open(f'{country}{ind}.pdf', "rb"))
            with open(f'Outputs/{country}/{country}_{data_year}.pdf', "wb") as pdfOutput:
                pdfMerge.write(pdfOutput)
            for ind in range(0, pageInfo['length']):
                os.remove(f'{country}{ind}.pdf')
